# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

You can use ** Keras ** to implement your model. Read more at [keras.io](https://keras.io/).

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0)). You are not expected to model your architecture precisely using this model nor get the same performance levels, but this is more to show an exampe of an approach used to solve this particular problem. We encourage you to try out different architectures for yourself and see what works best for you. Here is a useful [forum post](https://discussions.udacity.com/t/goodfellow-et-al-2013-architecture/202363) discussing the architecture as described in the paper and here is [another one](https://discussions.udacity.com/t/what-loss-function-to-use-for-multi-digit-svhn-training/176897) discussing the loss function.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [1]:

import logging
import time

import numpy
import tensorflow
from tensorflow.examples.tutorials.mnist import input_data

start_time = time.time()


def accuracy(arg_predictions, arg_labels):
    t0 = numpy.sum(numpy.argmax(arg_predictions, 1) == numpy.argmax(arg_labels, 1))
    result = (100.0 * t0 / arg_predictions.shape[0])
    return result


def weight_variable(arg_shape):
    distribution = tensorflow.truncated_normal(arg_shape, stddev=0.1)
    result = tensorflow.Variable(distribution)
    return result


def bias_variable(arg_shape):
    constant = tensorflow.constant(0.1, shape=arg_shape)
    result = tensorflow.Variable(constant)
    return result


def conv2d(arg_input, arg_filter):
    result = tensorflow.nn.conv2d(arg_input, arg_filter, strides=[1, 1, 1, 1], padding='SAME')
    return result


def max_pool(arg_value):
    result = tensorflow.nn.max_pool(arg_value, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return result


def model(arg_data, arg_weights1, arg_weights2, arg_weights3, arg_weights4, arg_bias1, arg_bias2, arg_bias3, arg_bias4,
          arg_keep_fraction):
    conv = conv2d(arg_data, arg_weights1)
    pool = max_pool(conv)
    hidden = tensorflow.nn.relu(pool + arg_bias1)
    conv = conv2d(hidden, arg_weights2)
    pool = max_pool(conv)
    hidden = tensorflow.nn.relu(pool + arg_bias2)
    shape = hidden.get_shape().as_list()
    reshape = tensorflow.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tensorflow.nn.relu(tensorflow.matmul(reshape, arg_weights3) + arg_bias3)
    drop = tensorflow.nn.dropout(hidden, arg_keep_fraction)
    result = tensorflow.matmul(drop, arg_weights4) + arg_bias4
    return result


logging.basicConfig(format='%(asctime)s : %(levelname)s :: %(message)s', level=logging.DEBUG)

mnist = input_data.read_data_sets("mnist_data/", one_hot=True)
data_train = numpy.reshape(mnist.train.images[10000:], [-1, 28, 28, 1])
labels_train = mnist.train.labels[10000:]
logging.debug('training data has shape %d x %d x %d x %d' % data_train.shape)
logging.debug('training labels have shape %d x %d' % labels_train.shape)
data_valid = numpy.reshape(mnist.train.images[:10000], [-1, 28, 28, 1])
labels_valid = mnist.train.labels[:10000]
logging.debug('validation data has shape %d x %d x %d x %d' % data_valid.shape)
logging.debug('validation labels have shape %d x %d' % labels_valid.shape)
data_test = numpy.reshape(mnist.test.images, [-1, 28, 28, 1])
labels_test = mnist.test.labels
logging.debug('test data has shape %d x %d x %d x %d' % data_test.shape)
logging.debug('test labels have shape %d x %d' % labels_test.shape)

batch_size = 32
channel_count = 1
depth = 16
drop_keep_fraction = 0.65
hidden_count = 128
image_height = 28
image_width = 28
label_count = 10
patch_size = 3  # was 5
random_seed = 1
step_size = 200
steps_limit = 10001

graph = tensorflow.Graph()
with graph.as_default():
    tensorflow.set_random_seed(random_seed)
    train_set = tensorflow.placeholder(tensorflow.float32, shape=(batch_size, image_height, image_width, channel_count))
    train_labels = tensorflow.placeholder(tensorflow.float32, shape=(batch_size, label_count))
    valid_set = tensorflow.constant(data_valid)
    test_set = tensorflow.constant(data_test)
    keep_fraction = tensorflow.placeholder(tensorflow.float32)

    weights_conv1 = weight_variable([patch_size, patch_size, channel_count, depth])
    bias_conv1 = bias_variable([depth])
    weights_conv2 = weight_variable([patch_size, patch_size, depth, depth])
    bias_conv2 = bias_variable([depth])
    weights_conv3 = weight_variable([image_height // 4 * image_width // 4 * depth, hidden_count])
    bias_conv3 = bias_variable([hidden_count])
    weights_conv4 = weight_variable([hidden_count, label_count])
    bias_conv4 = bias_variable([label_count])

    logits = model(train_set, weights_conv1, weights_conv2, weights_conv3, weights_conv4, bias_conv1, bias_conv2,
                   bias_conv3, bias_conv4, keep_fraction)
    loss = tensorflow.reduce_mean(tensorflow.nn.softmax_cross_entropy_with_logits(logits, train_labels))
    optimizer = tensorflow.train.GradientDescentOptimizer(0.05).minimize(loss)
    train_prediction = tensorflow.nn.softmax(logits)
    validation_logits = model(valid_set, weights_conv1, weights_conv2, weights_conv3, weights_conv4, bias_conv1,
                              bias_conv2, bias_conv3, bias_conv4, keep_fraction)
    valid_prediction = tensorflow.nn.softmax(validation_logits)
    test_logits = model(test_set, weights_conv1, weights_conv2, weights_conv3, weights_conv4, bias_conv1, bias_conv2,
                        bias_conv3, bias_conv4, keep_fraction)
    test_prediction = tensorflow.nn.softmax(test_logits)

with tensorflow.Session(graph=graph, config=tensorflow.ConfigProto(device_count={'GPU': 0})) as session:
    tensorflow.initialize_all_variables().run()
    logging.debug('Initialized all variables.')
    batch_data = []
    batch_labels = []
    for step in range(steps_limit):
        offset = (step * batch_size) % (labels_train.shape[0] - batch_size)
        batch_data = data_train[offset:(offset + batch_size), :, :, :]
        batch_labels = labels_train[offset:(offset + batch_size), :]
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict={train_set: batch_data,
                                                                                        train_labels: batch_labels,
                                                                                        keep_fraction: drop_keep_fraction})
        if step % step_size == 0:
            logging.debug('Minibatch loss at step %d: %f' % (step, l))
            logging.debug('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            feed_dict = {train_set: batch_data,
                         train_labels: batch_labels,
                         keep_fraction: 1.0}
            logging.debug(
                'Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(feed_dict=feed_dict), labels_valid))
    logging.info('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(feed_dict={train_set: batch_data, train_labels: batch_labels, keep_fraction: 1.0}),
        labels_test))

finish_time = time.time()
elapsed_hours, elapsed_remainder = divmod(finish_time - start_time, 3600)
elapsed_minutes, elapsed_seconds = divmod(elapsed_remainder, 60)
logging.info("Elapsed time: {:0>2}:{:0>2}:{:05.2f}".format(int(elapsed_hours), int(elapsed_minutes), elapsed_seconds))



Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


2017-04-12 20:41:41,248 : DEBUG :: training data has shape 45000 x 28 x 28 x 1
2017-04-12 20:41:41,249 : DEBUG :: training labels have shape 45000 x 10
2017-04-12 20:41:41,250 : DEBUG :: validation data has shape 10000 x 28 x 28 x 1
2017-04-12 20:41:41,251 : DEBUG :: validation labels have shape 10000 x 10
2017-04-12 20:41:41,251 : DEBUG :: test data has shape 10000 x 28 x 28 x 1
2017-04-12 20:41:41,252 : DEBUG :: test labels have shape 10000 x 10


Instructions for updating:
Use `tf.global_variables_initializer` instead.


2017-04-12 20:41:41,713 : WARNING :: From <ipython-input-1-a92f6ff2a47e>:116 in <module>.: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
2017-04-12 20:41:41,895 : DEBUG :: Initialized all variables.
2017-04-12 20:41:41,916 : DEBUG :: Minibatch loss at step 0: 2.495396
2017-04-12 20:41:41,917 : DEBUG :: Minibatch accuracy: 9.4%
2017-04-12 20:41:42,935 : DEBUG :: Validation accuracy: 11.4%
2017-04-12 20:41:44,681 : DEBUG :: Minibatch loss at step 200: 0.368229
2017-04-12 20:41:44,682 : DEBUG :: Minibatch accuracy: 93.8%
2017-04-12 20:41:45,677 : DEBUG :: Validation accuracy: 87.5%
2017-04-12 20:41:47,292 : DEBUG :: Minibatch loss at step 400: 0.214733
2017-04-12 20:41:47,293 : DEBUG :: Minibatch accuracy: 96.9%
2017-04-12 20:41:48,298 : DEBUG :: Validation accuracy: 92.0%
2017-04-12 20:41:49,923 : DEBUG :: Minibatch loss at step 600: 0.232038
201

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.